In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#import the necessary modules and functions

In [ ]:
!pip install sentinelhub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.4/220.4 KB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.2 MB/s eta 0:00:00
  Created wheel for sentinelhub: filename=sentinelhub-3.8.1-py3-none-any.whl size=244728 sha256=8cae638f90d7a98c13b816109a6ddb74a32ceecfecbefb7be39457243927855e
  Stored in directory: /root/.cache/pip/wheels/58/8f/04/66fbaf64ca4e1ce59add7facd3befbf4d4244d41281b979513
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.

In [ ]:
from sentinelhub import SHConfig

client_id = '758ff8a3-3d39-44cd-8a48-d54ec69da99e'
client_secret = '6MA||#*6&;li)cAdCGRxVYG76DF<0TD}UH|DNSq>'

In [ ]:
config = SHConfig()

if client_id and client_secret:
  config.sh_client_id = client_id
  config.sh_client_secret = client_secret

if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np


def plot_image(image, factor=1.0, clip_range=None, **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

#Coordinates of the AOI (Area Of Interest)

In [ ]:
salt_coords_wgs84 = [
  10.849685,
  33.315324,
  11.081773,
  33.452355
]
resolution = 60
salt_bbox = BBox(bbox=salt_coords_wgs84, crs=CRS.WGS84)
salt_size = bbox_to_dimensions(salt_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {salt_size} pixels")

Image shape at 60 m resolution: (355, 260) pixels


#Different scripts we'll be using

In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""


In [ ]:
evalscript_swir = """
//VERSION=3
function setup() {
  return {
    input: ["B12","B8A","B04", "dataMask"],
    output: { bands: 4 }
  };
}

function evaluatePixel(sample) {
  return [2.5 * sample.B12,2.5 * sample.B8A,2.5 * sample.B04, sample.dataMask ];
}
"""

In [ ]:
evalscript_barren_soil = """
//VERSION=3 (auto-converted from 1)

//Author: Monja Sebela

function evaluatePixel(s) {
    let val = 2.5 * ((s.B11 + s.B04)-(s.B08 + s.B02))/((s.B11 + s.B04)+(s.B08 + s.B02));
    return [2.5* val, s.B08, s.B11];
}
function setup() {
  return {
    input: [{
      bands: ["B02","B04","B08","B11","B12"]
    }],
    output: { bands: 3 }
  }
  """

In [ ]:
evalscript_ndvi = """
let ndvi = (B08 - B04) / (B08 + B04);

//Visualization, as used in EO Browser:
if (ndvi<-1.1) return [0,0,0];
else if (ndvi<-0.2) return [0.75,0.75,0.75];
else if (ndvi<-0.1) return [0.86,0.86,0.86];
else if (ndvi<0) return [1,1,0.88];
else if (ndvi<0.025) return [1,0.98,0.8];
else if (ndvi<0.05) return [0.93,0.91,0.71];
else if (ndvi<0.075) return [0.87,0.85,0.61];
else if (ndvi<0.1) return [0.8,0.78,0.51];
else if (ndvi<0.125) return [0.74,0.72,0.42];
else if (ndvi<0.15) return [0.69,0.76,0.38];
else if (ndvi<0.175) return [0.64,0.8,0.35];
else if (ndvi<0.2) return [0.57,0.75,0.32];
else if (ndvi<0.25) return [0.5,0.7,0.28];
else if (ndvi<0.3) return [0.44,0.64,0.25];
else if (ndvi<0.35) return [0.38,0.59,0.21];
else if (ndvi<0.4) return [0.31,0.54,0.18];
else if (ndvi<0.45) return [0.25,0.49,0.14];
else if (ndvi<0.5) return [0.19,0.43,0.11];
else if (ndvi<0.55) return [0.13,0.38,0.07];
else if (ndvi<0.6) return [0.06,0.33,0.04];
else return [0,0.27,0];
"""

In [ ]:
evalscript_waterquality = """
/*
Name:    Sentinel-2 Water Quality (Se2WaQ) 
Version: 1.0
Date:    2020-01-31
Author:      Nuno Sidónio Andrade Pereira
Affiliation: Polytechnic Institute of Beja, Portugal
License:     Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)
Refs.: [1]M. Potes et al., “Use of Sentinel 2 – MSI for water quality 
          monitoring at Alqueva reservoir, Portugal,” Proc. Int. Assoc. 
          Hydrol. Sci., vol. 380, pp. 73–79, Dec. 2018.
       [2]K. Toming, T. Kutser, A. Laas, M. Sepp, B. Paavel, and T. Nõges,
          “First Experiences in Mapping Lake Water Quality Parameters with
          Sentinel-2 MSI Imagery,” Remote Sens., vol. 8, no. 8, p. 640, 
          Aug. 2016.
*/

// user defined FLAGs
var FLAGparam = 5;
var FLAGbackGround = 2;

// Water-land contrast index (to define the background)
var NDWI = index(B03, B08); 

// Background indexes                           
var Black = [0];                                       // FLAGbackGround = 0
var NDVI = index(B08, B04);                            // FLAGbackGround = 1
var TrueColor = [B04*2.5, B03*2.5, B02*2.5];           // FLAGbackGround = 2

// Empirical models
var Chl_a = 4.26 * Math.pow(B03/B01, 3.94);            // FLAGparam = 0; S2-L2A; [1] Unit: mg/m3;        
var Cya = 115530.31 * Math.pow(B03 * B04 / B02, 2.38); // FLAGparam = 1; S2-L2A; [1] Unit: 10^3 cell/ml; 
var Turb = 8.93 * (B03/B01) - 6.39;                    // FLAGparam = 2; S2-L2A; [1] Unit: NTU;          
var CDOM = 537 * Math.exp(-2.93*B03/B04);              // FLAGparam = 3; S2-L1C; [2] Unit: mg/l;         
var DOC = 432 * Math.exp(-2.24*B03/B04);               // FLAGparam = 4; S2-L1C; [2] Unit: mg/l;         
var Color = 25366 * Math.exp(-4.53*B03/B04);           // FLAGparam = 5; S2-L1C; [2] Unit: mg.Pt/l;      

// Numerical values for the scales of parameters
var scaleChl_a = [0, 6, 12, 20, 30, 50];
var scaleCya   = [0, 10, 20, 40, 50, 100];
var scaleTurb  = [0, 4, 8, 12, 16, 20];
var scaleCDOM  = [0, 1, 2, 3, 4, 5];
var scaleDOC   = [0, 5, 10, 20, 30, 40];
var scaleColor = [0, 10, 20, 30, 40, 50];

// Colors for the scales
var s = 255;
var colorScale = 
  [
   [73/s, 111/s, 242/s],
   [130/s, 211/s, 95/s],
   [254/s, 253/s, 5/s],
   [253/s, 0/s, 4/s],
   [142/s, 32/s, 38/s],
   [217/s, 124/s, 245/s]
  ];

// Image generation
if (NDWI<0) {
  if ( FLAGbackGround == 0 ) {
    return Black;
  } else if ( FLAGbackGround == 1 ) {
    return [0, .5*(NDVI+1), 0];
  } else if ( FLAGbackGround == 2 ) {
    return TrueColor;
  }
} else {
  switch ( FLAGparam ) {
    case 0:
     return colorBlend(Chl_a, scaleChl_a, colorScale);
     break;
    case 1:
      return colorBlend(Cya, scaleCya, colorScale);
      break;
    case 2:
      return colorBlend(Turb, scaleTurb, colorScale);
      break;
    case 3:
      return colorBlend(CDOM, scaleCDOM, colorScale);
      break;
    case 4:
      return colorBlend(DOC, scaleDOC, colorScale);
      break;
    case 5:
      return colorBlend(Color, scaleColor, colorScale);
      break;
    default:
      return TrueColor;
  }
}
"""

In [ ]:
evalscript_ndmi = """
if (B8A == 0 || B11 == 0){
  return [0,0,0];
} else {
  var val = (B8A - B11)/(B8A + B11);
  
  var vmin = -0.8;
  var vmax = 0.8;
  var dv = vmax - vmin;
  
  var r = 0.0;
  var g = 0.0;
  var b = 0.0;

  
  var v = val;

  if (v < vmin){
    v = vmin;
  }
  if (v > vmax){
    v = vmax;
  }
  
  var l1 = 0.35;
  var l2 = 0.48;
  var l3 = 0.52;
  var l4 = 0.65;
  
  var level1 = (vmin + l1 * dv);
  var level2 = (vmin + l2 * dv);
  var level3 = (vmin + l3 * dv);
  var level4 = (vmin + l4 * dv);

  if (v < level1){
     r = 0.5 +  (v - vmin) / (level1 - vmin) / 2;
  } else if (v < level2) {
     r = 1;
     g = (v - level1) / (level2 - level1);
     b = 0;
  } else if (v < level3) {
     r = 1 + (level2 - v) / (level3 - level2);
     g = 1;
     b = (v - level2) / (level3 - level2);
  } else if (v < level4) {
     r = 0;
     g = 1 + (level3 - v) / (level4 - level3);
     b = 1;
  } else {
     b = 1.0 + (level4 - v) / (vmax - level4) / 2;
  }

   return [r, g, b];
  
}
"""

In [ ]:
evalscript_wetlands = """
// Detecting the Presence of Water in Wetlands with Sentinel-2 Satellite (abbrv. WIW)
//
// General formula: IF B8A<0.1804 AND B12<0.1131 THEN Water ELSE NoWater
//
// URL https://www.indexdatabase.de/db/xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx//

return B8A<0.1804&&B12<0.1131?[51/255,68/255,170/255]:[B04*5,B03*5,B02*5];

// colorBlend will return a blue color when surface water is detected, and lighten to a natural color when no water is detected
"""

In [ ]:
evalscript_ndwi = """
//VERSION=3
//This script was converted from v1 to v3 using the converter API

//ndwi
var colorRamp1 = [
  	[0, 0xFFFFFF],
  	[1, 0x008000]
  ];
var colorRamp2 = [
  	[0, 0xFFFFFF],
  	[1, 0x0000CC]
  ];

let viz1 = new ColorRampVisualizer(colorRamp1);
let viz2 = new ColorRampVisualizer(colorRamp2);

function evaluatePixel(samples) {
  var val = index(samples.B03, samples.B08);

  if (val < -0) {
    return viz1.process(-val);
  } else {
    return viz2.process(Math.sqrt(Math.sqrt(val)));
  }
}

function setup() {
  return {
    input: [{
      bands: [
        "B03",
        "B08"
      ]
    }],
    output: {
      bands: 3
    }
  }
}
"""

In [ ]:
evalscript_ndsi = """
////VERSION=3 (auto-converted from 1)
// MODIS Normalized Difference Salinity Index (NDSI)
// https://www.indexdatabase.de/db/si-single.php?rsindex_id=57=&sensor_id=14


let viz = ColorGradientVisualizer.createWhiteGreen(-0.89, 0.89);

function evaluatePixel(samples) {
    let val = index(samples.B06, samples.B07);
    return [...viz.process(val),samples.dataMask];
}

function setup() {
  return {
    input: [{
      bands: [ "B06", "B07", "dataMask" ]
    }],
    output: { bands: 4 }  }
}
"""

In [ ]:
evalscript_evap = """
//VERSION=3
function setup() 
{
    return {
        input: ["B11","B09","B02", "dataMask"],
        output: { bands: 4 }
        };
 }
        
function evaluatePixel(sample) {  
    return [2.5 * sample.B11,2.5 * sample.B09,2.5 * sample.B02, sample.dataMask ];
    }
  """

#sentinelhub-request

In [ ]:
from datetime import date
List = []
d = date(2002, 12, 1)
f = date(2002, 12, 27)
for j in range(2020,2023,1):
  d=d.replace(year = j)
  f=f.replace(year = j)
  for i in range(1,12,2):
    t0= str(d.replace(month = i))
    t1 = str(d.replace(month = i+1))
    List.append(t0)
    List.append(t1)

List.append('2023-01-01') 
print(List)

['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01']


In [ ]:
# sentinelhub request to know that you're using process api 
i=0
j=1

print( "The output directory has been created \n")
while j<len(List):
  request_code = SentinelHubRequest(
      data_folder="drive/MyDrive/dir_evap60",
      evalscript=evalscript_evap,
      input_data=[
          SentinelHubRequest.input_data(
              data_collection=DataCollection.SENTINEL2_L2A,                      #L2A is way better than L1C because of atmospheric correction
              time_interval=(List[i], List[j]),
              mosaicking_order=MosaickingOrder.LEAST_CC,
          )
      ],
      responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
      bbox=salt_bbox,
      size=salt_size,
      config=config,
  )

  i+=1
  j+=1

  print( "TIFF file with all needed bands was saved \n")
  #Saving your img
  result_imgs = request_code.get_data(save_data=True)

The output directory has been created 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was 

In [ ]:
!zip -r drive/MyDrive/dir_evap60/dir_evap60.zip drive/MyDrive/dir_evap60

  adding: drive/MyDrive/dir_evap60/ (stored 0%)
  adding: drive/MyDrive/dir_evap60/2a5e19f9e1834ff228bae36ad9042dd8/ (stored 0%)
  adding: drive/MyDrive/dir_evap60/2a5e19f9e1834ff228bae36ad9042dd8/response.tiff (deflated 0%)
  adding: drive/MyDrive/dir_evap60/2a5e19f9e1834ff228bae36ad9042dd8/request.json (deflated 64%)
  adding: drive/MyDrive/dir_evap60/6af5390bf601b7493a685791c17e6519/ (stored 0%)
  adding: drive/MyDrive/dir_evap60/6af5390bf601b7493a685791c17e6519/response.tiff (deflated 0%)
  adding: drive/MyDrive/dir_evap60/6af5390bf601b7493a685791c17e6519/request.json (deflated 64%)
  adding: drive/MyDrive/dir_evap60/6143a0173b4edc5f66ab061d3d7d5f35/ (stored 0%)
  adding: drive/MyDrive/dir_evap60/6143a0173b4edc5f66ab061d3d7d5f35/response.tiff (deflated 0%)
  adding: drive/MyDrive/dir_evap60/6143a0173b4edc5f66ab061d3d7d5f35/request.json (deflated 64%)
  adding: drive/MyDrive/dir_evap60/fa7f61b33e950ad92dd55c9dfa7b7395/ (stored 0%)
  adding: drive/MyDrive/dir_evap60/fa7f61b33e950ad92

In [ ]:
evalscript = """
//VERSION=3

function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: { bands: 3 }
  };
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
}
"""
request = SentinelHubRequest(
    data_folder="dir_barren_soil",
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.MODIS,          
            time_interval=('2022-12-01', '2023-01-01'),          
        ),
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.JPG),
    ],
    bbox=salt_bbox,
    size=salt_size,
    config=config,
)
result_imgs = request.get_data(save_data=True)

UnidentifiedImageError: ignored

In [ ]:
# sentinelhub request to know that you're using process api 
i=0
j=1

print( "The output directory has been created \n")
while j<len(List):
  request_code = SentinelHubRequest(
      data_folder="dir_ndsi60",
      evalscript=evalscript_ndsi,
      input_data=[
          SentinelHubRequest.input_data(
              data_collection=DataCollection.MODIS,                      #L2A is way better than L1C because of atmospheric correction
              time_interval=(List[i], List[j]),
          )
      ],
      responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
      bbox=salt_bbox,
      size=salt_size,
      config=config,
  )

  i+=1
  j+=1

  print( "TIFF file with all needed bands was saved \n")
  #Saving your img
  result_imgs = request_code.get_data(save_data=True)

The output directory has been created 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was saved 

TIFF file with all needed bands was 

In [ ]:
from google.colab import files
files.download('/content/dir_ndvi.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>